In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, recall_score, precision_score
from itertools import product
from joblib import Parallel, delayed

In [ ]:
path = '..\data\parkinsons.data'
data = pd.read_csv(path, sep=',')

In [ ]:
X = data.drop(columns=['name', 'status'])
y = data['status']

HOLD - OUT

In [ ]:
# Chạy 1500 tổ hợp: 10 x 15 x 10
param_grid = {
    'max_depth': list(range(3, 11)),
    'min_samples_split': list(range(5, 20)),
    'min_samples_leaf': list(range(2, 11)),
}

num_iterations = 100  # Lặp 100 lần mỗi tổ hợp
random_state_base = 42

combos = []
for max_depth, min_samples_split, min_samples_leaf in product(
    param_grid['max_depth'],
    param_grid['min_samples_split'],
    param_grid['min_samples_leaf']
):
    if min_samples_leaf >= min_samples_split:
        continue
    combos.append({
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf
    })


def evaluate_combo(params):
    f1_scores = []
    precision_scores = []
    recall_scores = []

    rskf = StratifiedShuffleSplit(n_splits=num_iterations, test_size=0.2, random_state=42)
    for split_id, (train_idx, test_idx) in enumerate(rskf.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = DecisionTreeClassifier(
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            random_state=random_state_base + split_id,
            criterion='gini',
            max_features='sqrt',
            class_weight='balanced',
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        f1_scores.append(f1_score(y_test, y_pred, average='binary'))
        precision_scores.append(precision_score(y_test, y_pred, average='binary'))
        recall_scores.append(recall_score(y_test, y_pred, average='binary'))

    return {
        **params,
        'mean_f1': np.mean(f1_scores),
        'std_f1': np.std(f1_scores),
        'mean_precision': np.mean(precision_scores),
        'std_precision': np.std(precision_scores),
        'mean_recall': np.mean(recall_scores),
        'std_recall': np.std(recall_scores)
    }

# Tính toán
results = Parallel(n_jobs=-1)(
    delayed(evaluate_combo)(params) for params in combos
)

# Kết quả
df_result = pd.DataFrame(results)
df_result_sorted = df_result.sort_values(by='mean_f1', ascending=False)

In [ ]:
df_result_sorted